In [2]:
# copyright ############################### #
# This file is part of the Xtrack Package.  #
# Copyright (c) CERN, 2021.                 #
# ######################################### #


import ducktrack as dtk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from tqdm import tqdm

import xtrack as xt
import xobjects as xo
import xpart as xp


beta_rel = 0.305
#beta_rel = 0.106

gamma = 1.050
#gamma = 1.006


current=2.4
length = 1.5 # m cooler length
r_beam=25*1e-3

mass0=938.27208816*1e6 #ev/c^2

T_perp = 100e-3 # <E> [eV] = kb*T
T_l =  1e-3 # <E> [eV]
magnetic_field = 0.060 # T for AD
B_ratio=1e-10



c=299792458.0

p0c = mass0*beta_rel*gamma #eV/c



circumference = 182.43280000000 #m
T = circumference/(c*beta_rel)
s_per_turn = T


beta_x=10 
beta_y=4

#disp_x=0.12

                                        


Parameters: $I=2.4 A, B=0.060 T, \beta_x=10 m,\beta_y=4,D_x=0.12 m T_\perp=100meV,T_\parallel=1meV,\epsilon_{x}=35*1e-6,\epsilon_{y}=35*1e-6,dp/p=1e-3$

In [3]:
import numpy as np
from scipy.optimize import minimize
from pyswarm import pso
from deap import algorithms, base, creator, tools

delta_single = 5e-4
num_turns = int(1e4)

# Define the objective function to minimize
def objective_function(params):
    disp_x, offset_x = params
    
    particles = xp.Particles(
        mass0=mass0,
        p0c=p0c,
        x=disp_x * delta_single,
        px=0,
        y=0,
        py=0,
        delta=delta_single,
        zeta=0
    )

    arc = xt.LinearTransferMatrix(
        Q_x=5.44, Q_y=5.42,
        beta_x_0=beta_x, beta_x_1=beta_x,
        beta_y_0=beta_y, beta_y_1=beta_y,
        alpha_x_0=0, alpha_x_1=0,
        alpha_y_0=0, alpha_y_1=0,
        disp_x_0=disp_x, disp_x_1=disp_x,
        disp_y_0=0, disp_y_1=0,
        beta_s=1 * 1e40,
        Q_s=0,
        chroma_x=0.0, chroma_y=0
    )

    cooler = xt.ElectronCooler(
        current=current,
        length=length,
        r_beam=r_beam,
        T_perp=T_perp,
        T_l=T_l,
        magnetic_field=magnetic_field,
        B_ratio=1e-4,
        Neutralisation_space_charge=1,
        offset_dp=0,
        offset_x=offset_x
    )

    action_x = []
    delta = []

    for turn in range(num_turns):
        action_x_temp = (particles.x ** 2 / beta_x + beta_x * particles.px ** 2)
        action_x.append(action_x_temp)
        delta.append(particles.delta)

        arc.track(particles)
        cooler.track(particles)

    action_reduction = action_x[-1] 
    delta_reduction = delta[-1] / delta[0]

    # Minimize the action reduction
    return action_reduction

def callback(xk):
    # Print current iteration and corresponding objective function value
    iteration = callback.iteration
    obj_value = objective_function(xk)
    print(f"Iteration: {iteration}, Objective Value: {obj_value}")
    callback.iteration += 1

# Genetic Algorithm (GA)
def ga_optimization():
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("disp_x", np.random.uniform, -20, 20)
    toolbox.register("offset_x", np.random.uniform, -5e-4, 5e-4)
    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.disp_x, toolbox.offset_x), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("evaluate", objective_function)

    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, halloffame=hof, verbose=True)



ModuleNotFoundError: No module named 'pyswarm'